### Tackle domain 2 classification (imbalanced learning)

In [1]:
import numpy as np

In [2]:
from scipy import sparse
temp_sparse_ = sparse.load_npz( "domain2_X_y_csr.npz" )
temp_loaded = temp_sparse_.toarray()
n_samples, n_features = temp_loaded.shape
n_features -= 1 #Since the last column is actually the label
X = temp_loaded[:,:n_features]
y = temp_loaded[:,n_features]
del temp_loaded, temp_sparse_

In [3]:
X.shape

(13000, 7755)

In [61]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN( random_state=0, n_jobs=-1, sampling_strategy = "majority" )

In [70]:
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.linear_model import LogisticRegressionCV

In [71]:
selector_percentile = SelectPercentile( f_classif, percentile = 50 )
X_new = selector_percentile.fit_transform( X, y )

/home/jvillanueva/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   15  159  363  718 1061 1145 1417 1582 1606 1748 2112 2302 3144 3309
 3477 4170 5272 5303 5598 5869 6143 6374 6770 6912 7170] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/jvillanueva/.local/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [72]:
X_resampled, y_resampled = smote_enn.fit_resample(X_new, y)
X_resampled.shape

InvalidParameterError: The 'sampling_strategy' parameter of SMOTEENN must be a float in the range (0.0, 1.0], a str among {'auto', 'minority', 'not minority', 'all', 'not majority'}, an instance of 'collections.abc.Mapping' or a callable. Got 'majority' instead.

In [73]:
unique, counts = np.unique(y_resampled, return_counts=True)
dict(zip(unique, counts))

{0.0: 7615, 1.0: 11459}

In [74]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

In [48]:
%%capture output1
clf = LogisticRegressionCV(cv=3, random_state=0, solver='newton-cholesky', n_jobs=2, class_weight="balanced")
clf.fit(X_resampled, y_resampled)
clf.score(X_resampled, y_resampled)
# clf = LogisticRegression(random_state=0, solver='newton-cholesky', n_jobs=-1, class_weight="balanced")
# clf.fit(X_resampled, y_resampled)

In [52]:
output1()

1.0

In [53]:
import pickle as pkl

In [54]:
with open("log_res_domain2.mdl", "wb") as f:
    pkl.dump( [clf,selector_percentile], f )